In [ ]:
BUBBLE_CHART_TOP_N = 6

In [ ]:
%run database.ipynb
%run pathutils.ipynb
%run export.ipynb

In [ ]:
# Load and preview the data
connection = connect()
retailer_stats_df = load_retailer_statistics(connection)

# Calculate the derived costs
retailer_stats_df["Cost_Per_Album"] = retailer_stats_df["Spend"] / retailer_stats_df["Albums"]
retailer_stats_df["Cost_Per_Track"] = retailer_stats_df["Spend"] / retailer_stats_df["Tracks"]
retailer_stats_df["Cost_Per_Artist"] = retailer_stats_df["Spend"] / retailer_stats_df["Artists"]

# Calculate the "breadth vs depth" and track density metrics
retailer_stats_df["Albums_Per_Artist"] = retailer_stats_df["Albums"] / retailer_stats_df["Artists"]
retailer_stats_df["Tracks_Per_Album"] = retailer_stats_df["Tracks"] / retailer_stats_df["Albums"]

display(retailer_stats_df)

In [ ]:
# Direct from artist vs retailers
direct_df = retailer_stats_df.groupby("ArtistDirect").agg({
    "Spend": "sum",
    "Albums": "sum",
    "Tracks": "sum"
})

display(direct_df)

In [ ]:
# Export the data
export_to_spreadsheet("retailer-statistics", {
    "Retailer Statistics": retailer_stats_df,
    "Artist Direct": direct_df
})

# Spend by Retailer

In [ ]:
import matplotlib.pyplot as plt

df = retailer_stats_df.copy()

df = df.sort_values("Spend", ascending=True)

plt.figure(figsize=(12,5))
plt.barh(df["Name"], df["Spend"])
plt.xlabel("Spend")
plt.title("Spend by retailer (sorted)")
plt.tight_layout()

# Export the chart
export_chart("spend-by-retailer", "", "png")

plt.show()

# Spend Pareto by Retailer

In [ ]:
import matplotlib.pyplot as plt

df = retailer_stats_df.copy().sort_values("Spend", ascending=False).reset_index(drop=True)

df["Spend_Pct"] = df["Spend"] / df["Spend"].sum()
df["Cumulative_Spend_Pct"] = df["Spend_Pct"].cumsum()

fig, ax1 = plt.subplots(figsize=(12, 5))
ax1.bar(df["Name"], df["Spend"])
ax1.set_ylabel("Spend")
ax1.set_title("Spend Pareto by retailer")
ax1.tick_params(axis="x", rotation=90)

ax2 = ax1.twinx()
ax2.plot(df["Name"], df["Cumulative_Spend_Pct"], color="orange", linestyle="--")
ax2.set_ylabel("Cumulative Share of Spend")
ax2.set_ylim(0, 1.05)

fig.tight_layout()

# Export the chart
export_chart("spend-pareto-by-retailer", "", "png")

plt.show()

# Spend - Artist vs Retailer

In [ ]:
import matplotlib.pyplot as plt

d = direct_df.copy()

# Ensure index is 0/1 and in a sensible order
d = d.sort_index()

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(d.index.astype(str), d["Spend"])
ax.set_xlabel("Supplier (0 = Retailer, 1 = Artist)")
ax.set_ylabel("Spend")
ax.set_title("Total Spend: Retailer vs Direct from Artist")
fig.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(d.index.astype(str), d["Albums"])
ax.set_xlabel("Supplier (0 = Retailer, 1 = Artist)")
ax.set_ylabel("Albums")
ax.set_title("Total Albums: Retailer vs Direct from Artist")
fig.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(d.index.astype(str), d["Tracks"])
ax.set_xlabel("Supplier (0 = Retailer, 1 = Artist)")
ax.set_ylabel("Tracks")
ax.set_title("Total Tracks: Retailer vs Direct from Artist")
fig.tight_layout()

# Export the chart
export_chart("spend-artist-vs-retailer", "", "png")

plt.show()

# Cost per Album - Artist vs Retailer

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

d = direct_df.copy().sort_index()

d["Cost_Per_Album"] = d["Spend"] / d["Albums"].replace(0, np.nan)
d["Cost_Per_Track"] = d["Spend"] / d["Tracks"].replace(0, np.nan)

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(d.index.astype(str), d["Cost_Per_Album"])
ax.set_xlabel("Supplier (0 = Retailer, 1 = Artist)")
ax.set_ylabel("Cost per Album")
ax.set_title("Cost per Album: Retailer vs Direct from Artist")
fig.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(d.index.astype(str), d["Cost_Per_Track"])
ax.set_xlabel("Supplier (0 = Retailer, 1 = Artist)")
ax.set_ylabel("Cost per Track")
ax.set_title("Cost per Track: Retailer vs Direct from Artist")
fig.tight_layout()

# Export the chart
export_chart("cost-per-album-artist-vs-retailer", "", "png")

plt.show()

# Retailer Breadth vs Depth

In [ ]:
import matplotlib.pyplot as plt

df = retailer_stats_df.copy()

# Bubble size scaling
size = df["Spend"].to_numpy()
s = 50 + 450 * (size - size.min()) / (size.max() - size.min() + 1e-9)

plt.figure(figsize=(12, 5))
plt.scatter(df["Artists"], df["Albums_Per_Artist"], s=s, alpha=0.7)
plt.xlabel("Artists (breadth)")
plt.ylabel("Albums per artist (depth)")
plt.title("Breadth vs depth (labels: top spend + notable)")

# Label rules: top N spend + any with high breadth or high depth
top_spend = df.nlargest(BUBBLE_CHART_TOP_N, "Spend").index
notable = df[(df["Artists"] >= df["Artists"].quantile(0.85)) |
            (df["Albums_Per_Artist"] >= df["Albums_Per_Artist"].quantile(0.85))].index
label_idx = top_spend.union(notable)

for i, r in df.loc[label_idx].iterrows():
    plt.text(r["Artists"], r["Albums_Per_Artist"], r["Name"], fontsize=8, ha="left", va="bottom")

plt.tight_layout()

# Export the chart
export_chart("retailer-breadth-depth", "", "png")

plt.show()

# Cost per Album - Artist vs Retailer

In [ ]:
import matplotlib.pyplot as plt

df = retailer_stats_df.copy()

# Boxplot wants arrays in order
groups = [
    df.loc[df["ArtistDirect"] == 0, "Cost_Per_Album"].dropna().to_numpy(),
    df.loc[df["ArtistDirect"] == 1, "Cost_Per_Album"].dropna().to_numpy(),
]

plt.figure(figsize=(12, 5))
plt.boxplot(groups, tick_labels=["Retailer", "Artist"])
plt.ylabel("Cost per Album")
plt.title("Cost per Album Distribution: Retailer vs Direct from Artist")
plt.tight_layout()

# Export the chart
export_chart("cost-per-album-artist-vs-retailer-box-chart", "", "png")

plt.show()